Site Scraper

In [ ]:
import requests
import urllib.request
from urllib.request import Request, urlopen
import pandas as pd
import re
from bs4 import BeautifulSoup
#gotta use selenium ig L9 
abbrev_names = {'AtlantaHawks': 'ATL', 'ATLHawks': 'ATL', 'BostonCeltics': 'BOS', 'BOSCeltics': 'BOS',
     'BrooklynNets': 'BKN', 'BKNNets': 'BKN', 'CharlotteHornets': 'CHA', 'CHAHornets': 'CHA',
     'ChicagoBulls': 'CHI', 'CHIBulls': 'CHI', 'DallasMavericks': 'DAL', 'DALMavericks': 'DAL',
     'DenverNuggets': 'DEN', 'DENNuggets': 'DEN', 'DetroitPistons': 'DET', 'DETPistons': 'DET',
     'GoldenStateWarriors': 'GSW', 'GSWWarriors': 'GSW', 'GSWarriors': 'GSW', 'HoustonRockets': 'HOU', 'HOURockets': 'HOU',
     'IndianaPacers': 'IND', 'INDPacers': 'IND', 'LosAngelesClippers': 'LAC', 'LAClippers': 'LAC',
     'LosAngelesLakers': "LAL", 'LALakers': 'LAL', 'MemphisGrizzlies': 'MEM', 'MEMGrizzlies': 'MEM',
     'MiamiHeat': 'MIA', 'MIAHeat': 'MIA', 'MilwaukeeBucks': 'MIL', 'MILBucks': 'MIL', 
     'MinnesotaTimberwolves': 'MIN', 'MINTimberwolves': 'MIN', 'NewOrleansPelicans': 'NOP',
     'NOPelicans': 'NOP', 'NewYorkKnicks': 'NYK', 'NYKnicks': 'NYK', 'OklahomaCityThunder': 'OKC',
     'OKCThunder': 'OKC', 'OrlandoMagic': 'ORL', 'ORLMagic': 'ORL', 'Philadelphia76ers': 'PHI',
     'PHI76ers': 'PHI', 'PhoenixSuns': 'PHX', 'PHXSuns': 'PHX', 'PHOSuns': 'PHX', 'PortlandTrailBlazers': 'POR',
     'PORTrailBlazers': 'POR', 'SacramentoKings': 'SAC', 'SACKings': 'SAC', 'SanAntonioSpurs': 'SAS',
     'SASpurs': 'SAS', 'TorontoRaptors': 'TOR', 'TORRaptors': 'TOR', 'UtahJazz': 'UTA', 'UTJazz': 'UTA',
     'WashingtonWizards': 'WAS', 'WASWizards': 'WAS'}

#sportsbetting website
URL = "https://www.sportsbetting.ag/sportsbook/basketball/nba"
URL1 = "https://sportsbook.draftkings.com/leagues/basketball/88670846"
URL2 = "https://sportsbook.fanduel.com/navigation/nba"



#fake header to get data from site 
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(URL,headers=hdr)
page = urlopen(req)

req1 = Request(URL1,headers=hdr)
page1 = urlopen(req1)

req2 = Request(URL2,headers=hdr)
page2 = urlopen(req2)

#HTML source
soup = BeautifulSoup(page)
soup1 = BeautifulSoup(page1)
soup2 = BeautifulSoup(page2)
#print(soup2.prettify())

#scrape data
teamsHTML = soup.find_all('td', attrs={'class': 'col_teamname bdevtt'})
oddsHTML = soup.find_all('td', attrs={'class': 'odds bdevtt moneylineodds displayOdds'})

teamsHTML1 = soup1.find_all('div', attrs={'class': 'event-cell__name-text'})
oddsHTML1 = soup1.find_all('span', attrs={'class': 'sportsbook-odds american no-margin default-color'})

teamsHTML2 = soup2.find_all('span', attrs={'class': 's x jk jl jm jn ht hu hv jc jo h fn dh jp bc'})
oddsHTML2 = soup2.find_all('span', attrs={'id': 'ctl00_ctl00_M_middle_ConstructorLines1_GameLines1_repHeaders_ctl00_repLines_ctl00_lblBetVisitorMoneyLine'})

print(teamsHTML2)
print(oddsHTML2)

teams = []
odds = []
teams1 = []
odds1 = []

#american odds to decimal odds
def dec_odds(odd):
  res = []
  for k in range(len(odd)):
    if(int(odd[k])>=0):
      res.append(int(odd[k])/100+1)
    else:
      res.append(1-(100/int(odd[k])))
  return res

#clean team names data
def clean_teams(ht):
  res = []
  for h in ht:
    res.append(abbrev_names[h.getText().replace("\n","").replace("\t","").replace(" ","")])
  return res

#clean odds data
def clean_odds(odd):
  res = []
  for o in odd:
    res.append(o.getText().replace("\n","").replace("\t",""))
  return res

teams = clean_teams(teamsHTML)
odds = clean_odds(oddsHTML)
teams1 = clean_teams(teamsHTML1)
odds1 = clean_odds(oddsHTML1)

print(teams)
print(odds)
print(teams1)
print(odds1)

#compile data from one site into dataframe
def site_data(t, o):
  rows = []
  for i in range(0,len(t), 2):
    row = []
    row.append(t[i])
    row.append(t[i+1])
    row.append(o[i])
    row.append(o[i+1])
    rows.append(row)
  return pd.DataFrame(columns=['Team 1', 'Team 2', 'T1 Odds', 'T2 Odds'], data=rows)

odds = dec_odds(odds)
odds1 = dec_odds(odds1)

df = site_data(teams, odds)
df1 = site_data(teams1, odds1)
print(df)
print(df1)



[]
[]
['BOS', 'GSW']
['+150', '-170']
['BOS', 'GSW']
['+145', '-165']
  Team 1 Team 2  T1 Odds   T2 Odds
0    BOS    GSW      2.5  1.588235
  Team 1 Team 2  T1 Odds   T2 Odds
0    BOS    GSW     2.45  1.606061


Arbitrage Calculation

In [ ]:
site1_data = ['Memphis Grizzlies ', 'Golden State Warriors ', 3.95, 1.274]
site2_data = ['Memphis Grizzlies ', 'Golden State Warriors ', 1.5, 2.8]

tennis1 = ['Djokovic', 'Medvedev', 1.9, 1.9]
tennis2 = ['Djokovic', 'Medvedev', 1.4, 2.5]

#function to find arbitrage possibility given site data
def arb():
  return 0